In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from tqdm import tqdm_notebook as tqdm
import re

## Scraping NYT pages for Articles given URL

We define a ```get_article``` function which scrapes the HTML of the NYTimes page, then given a text file of links from Twitter will run the function in a loop (which takes a good long while)

In [5]:
# source: https://towardsdatascience.com/a-practitioners-guide-to-natural-language-processing-part-i-processing-understanding-text-9f4abfd13e72
def remove_special_chars(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s{.}{;}]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

In [6]:
remove_special_chars("áaã.")

'a.'

In [13]:
# Some code pulled from: 
# https://stackoverflow.com/questions/40699015/python-beautifulsoup-new-york-times-web-article-scraping

def get_article(url):
    ''' if given the url for a specific nyt article, will grab the article text '''
    
    session = requests.Session()
    req = session.get(url)
    soup = BeautifulSoup(req.text)
    
    paragraphs = soup.find_all('p')
    article = ''
    for p in paragraphs:
        article = article + p.get_text()
    
    ### Clean up some
    
    # article = remove_special_chars(article)
    
    # source: https://stackoverflow.com/questions/199059/a-pythonic-way-to-insert-a-space-before-capital-letters
    article = re.sub(r"(\w)([A-Z])", r"\1 \2", article)
    
    try:
        article = article.split('\n', 1)[-1]
    except:
        pass
    
    try:
        article = article.split('A version of this article', 1)[0]
    except:
        pass
    
    try:
        article = article.split('Were interested in your feedback on this page', 1)[0]
    except:
        pass

    return article

In [16]:
get_article(urls[5010])

'\n      By T AR IR O M ZE ZE WA\n\nA UG. 5, 2019\n\nSand and the ocean aren’t for everyone. And that’s O.K.\nSand and the ocean aren’t for everyone. And that’s O.K.By T AR IR O M ZE ZE WA A UG. 5, 2019 One of the enduring pleasures of summer is a trip to the beach — the blankets and umbrellas, the briny air, the rhythmic water. Who among us doesn’t love digging our toes into the sand and lounging in the waves?Lots of people, it turns out. For many vacationers, the beach is an inconvenience: Sand gets everywhere, waves knock you down and the prospect of encountering one of the 2 million unknown species lurking in the water can be terrifying.For those who fear the ocean or who are unable to access the beach because of disabilities or medical reasons, pools can be an exciting alternative. And for some, being in a pool rather than the ocean simply feels safer because they can rest their feet on the bottom and see what’s down below.At Polihale Beach, there are no lifeguards, no potable wat

In [17]:
urls = [line.rstrip('\n') for line in open('/Users/Kyelee/Documents/Columbia/senior_seminar/bash_scripts/twitlinks.txt')]

In [18]:
len(urls)

11567

In [29]:
df = pd.DataFrame(index = range(len(urls)), columns = ['url', 'text'])
df.url = urls
n = 1

for i in tqdm(range(len(urls))):
    time.sleep(3)
    try:
        df.text[i] = get_article(urls[i])
    except:
        print("exception", n)
        n = n + 1
        pass
    
    
df.to_pickle("./text.pkl")

exception 1
exception 2
exception 3
exception 4
exception 5
exception 6
exception 7
exception 8
exception 9
exception 10
exception 11
exception 12
exception 13
exception 14
exception 15
exception 16
exception 17
exception 18
exception 19
exception 20
exception 21
exception 22
exception 23
exception 24
exception 25
exception 26
exception 27
exception 28
exception 29
exception 30
exception 31
exception 32
exception 33
exception 34
exception 35
exception 36
exception 37
exception 38
exception 39
exception 40
exception 41
exception 42
exception 43
exception 44
exception 45
exception 46
exception 47
exception 48
exception 49
exception 50
exception 51
exception 52
exception 53
exception 54
exception 55
exception 56
exception 57
exception 58
exception 59
exception 60
exception 61
exception 62
exception 63
exception 64
exception 65
exception 66
exception 67
exception 68
exception 69
exception 70
exception 71
exception 72
exception 73
exception 74
exception 75
exception 76
exception 77
exceptio

## Read the saved pickle file and CLEAN

In [30]:
unpickled_df = pd.read_pickle("./text.pkl")

In [31]:
unpickled_df

,url,text
0,https://nyti.ms/2MfGkpT,Advertisement Supported by As the cultural fir...
1,https://nyti.ms/2OMm5TN,Advertisement Supported by To conservatives in...
2,https://nyti.ms/2ISx9Lj,Advertisement Supported by Op-Docs One man’s u...
3,https://nyti.ms/2Mga6wf,Advertisement Supported by With a new “Addams ...
4,https://nyti.ms/2pSoGB5,"\n By A.O. S CO TT\n\nO CT. 8, 2019\n\nTh..."
...,...,...
11562,https://nyti.ms/2EK6vlI,Advertisement Supported by By Tiffany May HO N...
11563,https://nyti.ms/2ENHfLr,Advertisement Supported by On Soccer Jürgen Kl...
11564,https://nyti.ms/2EK5Nos,"Advertisement Supported by Every month, subscr..."
11565,https://nyti.ms/2EKlM68,Advertisement Supported by TI PS Hotels with p...


In [32]:
unpickled_df = unpickled_df.dropna()
print(unpickled_df.shape[0])

11470


In [34]:
unpickled_df.text[10]

'Advertisement Supported by By James Glanz and Brad Plumer This article was originally published on Oct. 12. It has been updated to reflect the latest developments.California has a reputation as a haven for technological innovation. But the state’s largest power utility is using the lowest of low-tech solutions — rolling blackouts — to protect dry landscapes from live power lines that could spark or overheat and set wildfires.Responding to forecasts of strong winds, the utility, Pacific Gas & Electric, expanded its blackout zone in Northern California on Saturday to include 940,000 customers — affecting as many as 2.7 million people — even as the local authorities ordered the evacuation of 50,000 residents because of the spreading Kincade fire. The outage comes on the heels of another major blackout by the utility earlier this month.There is technology, though, that can reduce the need for some of the most dangerous power lines and limit the extent of territory left in the dark.“It’s a

In [40]:
### CLEAN UP

def clean(text):
    try:
        text = text.split('Follow Us', 1)[0]
    except:
        pass
    
    try:
        text = text.split('Follow The New York Times', 1)[0]
    except:
        pass
    
    try:
        text = text.split('The Times is committed to publishing a diversity of letters to the editor', 1)[0]
    except:
        pass
    
    try:
        text = text.replace('Advertisement', '')
    except:
        pass
    
    try:
        text = text.split('An earlier version of this article', 1)[0]
    except:
        pass
    
    try:
        text = text.split('Reporting was contributed by', 1)[0]
    except:
        pass
    
    try:
        text = text.split('By', 1)[-1]
    except:
        pass
    
    try:
        text = text.replace('\n', ' ')
    except:
        pass
    
    # Keep only words
    # pattern = re.compile('(?<!\S)[A-Z][a-z]+(?!\S)|(?<!\S)[a-z]+(?!\S)|(?<!\S)[0-9]+(?!\S)')
    # text = " ".join(re.findall(pattern, text))
    
    return text 

unpickled_df['text'] = unpickled_df['text'].apply(clean)

/Users/Kyelee/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [41]:
unpickled_df

,url,text
0,https://nyti.ms/2MfGkpT,Cara Buckley AT LA NT A — Tyler Perry hears v...
1,https://nyti.ms/2OMm5TN,Robert Leonard Mr. Leonard is the author of “...
2,https://nyti.ms/2ISx9Lj,Shuhan Fan and Luther Clement Ms. Fan and Mr....
3,https://nyti.ms/2Mga6wf,Devin Fuller Twenty-six years after their las...
4,https://nyti.ms/2pSoGB5,"A.O. S CO TT O CT. 8, 2019 I spent my adoles..."
...,...,...
11562,https://nyti.ms/2EK6vlI,Tiffany May HO NG K ON G — For a few heady mo...
11563,https://nyti.ms/2ENHfLr,"the last day, I felt it was gone,” defender A..."
11564,https://nyti.ms/2EK5Nos,Jennifer Vineyard Watching is The New York Ti...
11565,https://nyti.ms/2EKlM68,"a miracle, I had booked us in a hotel next to..."


In [46]:
unpickled_df.to_pickle("../clean_text.pkl")

In [47]:
clean_df = pd.read_pickle("../clean_text.pkl")
clean_df

,url,text
0,https://nyti.ms/2MfGkpT,Cara Buckley AT LA NT A — Tyler Perry hears v...
1,https://nyti.ms/2OMm5TN,Robert Leonard Mr. Leonard is the author of “...
2,https://nyti.ms/2ISx9Lj,Shuhan Fan and Luther Clement Ms. Fan and Mr....
3,https://nyti.ms/2Mga6wf,Devin Fuller Twenty-six years after their las...
4,https://nyti.ms/2pSoGB5,"A.O. S CO TT O CT. 8, 2019 I spent my adoles..."
...,...,...
11562,https://nyti.ms/2EK6vlI,Tiffany May HO NG K ON G — For a few heady mo...
11563,https://nyti.ms/2ENHfLr,"the last day, I felt it was gone,” defender A..."
11564,https://nyti.ms/2EK5Nos,Jennifer Vineyard Watching is The New York Ti...
11565,https://nyti.ms/2EKlM68,"a miracle, I had booked us in a hotel next to..."


In [49]:
# clean_df.text[100]